In [5]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV 
import pandas as pd
import traceback
import talib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)





cycle_Indicators = ['HT_DCPERIOD', 'HT_DCPHASE',  'HT_TRENDMODE']


stadistics_indicator =  [ "TSF" , "LINEARREG_INTERCEPT"]

#TA indicators
indicators = [
        'CDL2CROWS',
        'CDL3BLACKCROWS',
        'CDL3INSIDE',
        'CDL3LINESTRIKE',
        'CDL3OUTSIDE',
        'CDL3STARSINSOUTH',
        'CDL3WHITESOLDIERS',
        'CDLABANDONEDBABY',
        'CDLADVANCEBLOCK',
        'CDLBELTHOLD',
        'CDLBREAKAWAY',
        'CDLCLOSINGMARUBOZU',
        'CDLCONCEALBABYSWALL',
        'CDLCOUNTERATTACK',
        'CDLDARKCLOUDCOVER',
        'CDLDOJI',
        'CDLDOJISTAR',
        'CDLDRAGONFLYDOJI',
        'CDLENGULFING',
        'CDLEVENINGDOJISTAR',
        'CDLEVENINGSTAR',
        'CDLGAPSIDESIDEWHITE',
        'CDLGRAVESTONEDOJI',
        'CDLHAMMER',
        'CDLHANGINGMAN',
        'CDLHARAMI',
        'CDLHARAMICROSS',
        'CDLHIGHWAVE',
        'CDLHIKKAKE',
        'CDLHIKKAKEMOD',
        'CDLHOMINGPIGEON',
        'CDLIDENTICAL3CROWS',
        'CDLINNECK',
        'CDLINVERTEDHAMMER',
        'CDLKICKING',
        'CDLKICKINGBYLENGTH',
        'CDLLADDERBOTTOM',
        'CDLLONGLEGGEDDOJI',
        'CDLLONGLINE',
        'CDLMARUBOZU',
        'CDLMATCHINGLOW',
        'CDLMATHOLD',
        'CDLMORNINGDOJISTAR',
        'CDLMORNINGSTAR',
        'CDLONNECK',
        'CDLPIERCING',
        'CDLRICKSHAWMAN',
        'CDLRISEFALL3METHODS',
        'CDLSEPARATINGLINES',
        'CDLSHOOTINGSTAR',
        'CDLSHORTLINE',
        'CDLSPINNINGTOP',
        'CDLSTALLEDPATTERN',
        'CDLSTICKSANDWICH',
        'CDLTAKURI',
        'CDLTASUKIGAP',
        'CDLTHRUSTING',
        'CDLTRISTAR',
        'CDLUNIQUE3RIVER',
        'CDLUPSIDEGAP2CROWS',
        'CDLXSIDEGAP3METHODS']


## Used for generating the variables
def get_X  (data_ , model_params, data=False ):
    periods = model_params["periods"]
    time_periods = model_params ["time_periods"]
   
    # It loads the data we get from Binance

    if data :
        df = pd.DataFrame(data_ , columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'quote_asset_volume','num_trades','taker_base_vol','taker_quote_vol', 'ignore'] )
        
    else: 
        df= pd.read_csv( f"{data_}" )
        df.drop("Unnamed: 0" , axis=1 , inplace=True)
    df = df.astype(float )
    df.sort_values(by="open_time")
    df["open_time" ] = pd.to_datetime(df["open_time" ] , utc=True, unit='ms')  
    df["Typical"] = df[['open', 'high', 'low', 'close'] ].mean(axis = 1)
    for indicator in indicators:
        df[indicator] =   getattr(talib , indicator)( df.open , df.high, df.low , df.close)
    for cyclic in cycle_Indicators:
        df[cyclic] = getattr(talib , cyclic) (df.close)
    for period in periods:
        for stadistics in stadistics_indicator:
            df[stadistics] = getattr(talib ,stadistics )(df.close , timeperiod=period)
        df["SMA_{}".format(period)] = talib.SMA(df["Typical"] ,  timeperiod=period )
        df["DEMA_{}".format(period)] = talib.SMA(df["Typical"] ,  timeperiod=period )
        df["WMA_{}".format(period)] = talib.WMA(df["close"], timeperiod=period)
        [ df["upperband"] , df["middleband"] , df["lowerband"]  ]  = talib.BBANDS(df["close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    #volumne indicators
    df["AD"] = talib.AD(  df["high"], df["low"], df["close"], df["volume"])
    df["OBV"] = getattr(talib , "OBV") (df.close , df.volume)
    df["ADOSC"] = talib.ADOSC(  df["high"], df["low"], df["close"], df["volume"] ,  fastperiod=3, slowperiod=10)
    #volatiliy
    df["ATR"] = talib.ATR(  df["high"], df["low"], df["close"], timeperiod=time_periods[1])
    df["NATR"] = talib.NATR(  df["high"], df["low"], df["close"], timeperiod=time_periods[1])
    df["TRANGE"] = talib.NATR(  df["high"], df["low"], df["close"])

    #momentum indicators
    df["RSI"] = talib.RSI( df.close, timeperiod=time_periods[1])
    df["ADX"] = talib.ADX( df.high , df.low , df.close, timeperiod=time_periods[2])
    df["ADXR"] = talib.ADXR( df.high , df.low , df.close, timeperiod=time_periods[2])
    df["APO"] = talib.APO(  df.close, fastperiod=time_periods[1] , slowperiod= time_periods[2])
    df["AROON_up"] , df["AROON_down"]  = talib.AROON(  df.high , df.low , timeperiod=time_periods[2])
    df["AROONOSC"] = talib.AROONOSC(  df.high , df.low , timeperiod=time_periods[2])
    df["BOP"] = talib.BOP(  df.open , df.high , df.low , df.close )
    df["CCI"] = talib.CCI(  df.high , df.low ,  df.close, timeperiod=time_periods[1])
    df["CMO"] = talib.CMO(    df.close, timeperiod=time_periods[1])
    df["DX"] = talib.DX(   df.high , df.low ,  df.close, timeperiod=time_periods[1])
    df["macd"], df["macdsignal"], df["macdhist"] = talib.MACD(df.close, fastperiod=time_periods[1], slowperiod=time_periods[2], signalperiod=time_periods[0])
    df["macd_"], df["macdsignal_"], df["macdhist_"] = talib.MACDFIX(df.close, signalperiod=time_periods[1])
    df["MFI"] = talib.MFI(df.high, df.low, df.close, df.volume, timeperiod=time_periods[2])
    df["MINUS_DI"]= talib. MINUS_DI(df.high, df.low,df. close, timeperiod=time_periods[2])
    df["close_time" ] = pd.to_datetime(df["close_time" ] , utc=True, unit='ms')  
    df.set_index( "open_time"  , inplace=True)
    df = df.dropna(axis=0)
    return df
# Used for classifing 
def get_df (data_ , model_params, data=False ) :
    percentage =model_params["percentage"]
    space  =model_params["space"]
    test  =model_params["test"]
    df = get_X (data_ , model_params, data=False )
    df["label"]= np.nan
    data = pd.DataFrame([])
    total = int(df.shape[0] / space+test)
    


    for i in range(total):

        try:
            expect = df.iloc[  (space+test)*i :  (space+test) *(i+1),: ].iloc[ -space: ,: ]     
            look = df.iloc[  (space+test)*i :  (space+test) *(i+1),: ].iloc[   :-test, :] 
            
            start =   float(look[ -1: ]["close"])
            low = expect["low"]
            high = expect["high"]

            for i in range (expect.shape[0]):

                try:
                
                    low_percentage = (   low -start )/ (start)
                    high_percentage = (high - start )/start 

                    shorts = low_percentage <=  - percentage
                    longs = high_percentage >= percentage
                    if (shorts.iloc[i]):
                        look.loc[ -1: ,"label"] = 2
                        data = pd.concat([ data  ,  look[ -1: ]])

                        break
                    if (longs.iloc[i]):
                        look.loc[ -1: ,"label"] = 1
                        data = pd.concat([ data  ,  look[ -1: ]])
                        break
                    if (i == expect.shape[0] -1 ):
                        look.loc[ -1: ,"label"] = 0
                        data = pd.concat([ data  ,  look[ -1: ]])
                        break
                except :
                    #traceback.print_exc()

                    pass
        except:
            #traceback.print_exc()
            pass
    return data
    


In [48]:
#Some parameters for calculating the TA metrics

model_params = {"periods":[ 5, 20 ,30 ],
"time_periods" : [5,20,30],
"periods" :[5,10,20],

"percentage" : 0.04,
"space" : 1,
"test" : 8 }



In [49]:
df = get_df("ETHUSDT",model_params)


/tmp/ipykernel_150929/230212208.py:135: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["macd_"], df["macdsignal_"], df["macdhist_"] = talib.MACDFIX(df.close, signalperiod=time_periods[1])
/tmp/ipykernel_150929/230212208.py:135: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["macd_"], df["macdsignal_"], df["macdhist_"] = talib.MACDFIX(df.close, signalperiod=time_periods[1])
/tmp/ipykernel_150929/230212208.py:135: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [19]:
df_plot = df
df_plot["short"] =np.where( df_plot["label"] ==2 , 1,0)
df_plot["long"] =np.where( df_plot["label"] ==1 , 1,0)
df_plot["hold"] =np.where( df_plot["label"] ==0 , 1,0)





In [66]:
import plotly.graph_objects as go 
from plotly.subplots import make_subplots 
 

 
 
# Create figure with secondary y-axis 
fig = make_subplots(specs=[[{"secondary_y": True}]]) 
 
 
# Add traces 
fig.add_trace( 
    go.Scatter(x=df_plot.index, y=df_plot["close"], name="close"), 
    secondary_y=False, 
) 
 
fig.add_trace( 
    go.Scatter(x=df_plot.index, y=df_plot["short"],  name="short"), 
    secondary_y=True, 
) 

fig.add_trace( 
    go.Scatter(x=df_plot.index, y=df_plot["long"],  name="long"), 
    secondary_y=True, 
) 
 
 

 
# Add figure title 
fig.update_layout( 
    title_text="Precio" 
) 
 
# Set x-axis title 
fig.update_xaxes(title_text="") 
 
# Set y-axes titles 
fig.update_yaxes(title_text="<b>tiempo de respuesta</b> (ms)", range=[1000,4000], secondary_y=False) 
fig.update_yaxes(title_text="<b>predicción</b> Error", secondary_y=True) 
fig.update_layout(plot_bgcolor = "white") 
 
 
fig.show()

In [54]:
### Split the tadabase by time in train test and Balance the data

from imblearn.over_sampling import SMOTE


def X_y_train_test( percentil , df):
    #df.reset_index(inplace=True,drop =True)


    X_train = df.iloc[ :int( df.shape[0] *percentil)].drop([ "label"  ] ,axis =1)
    Y_train = df.iloc[ :int( df.shape[0] *percentil)]["label"]
    X_test = df.iloc[ int( df.shape[0] *percentil):].drop([ "label" ] ,axis =1)
    Y_test = df.iloc[ int( df.shape[0] *percentil):]["label"]
    return [ X_train , X_test , Y_train ,   Y_test]


def estimate_metrics(params, df):

    data_ = df[df['label'].notna()]
    data_.reset_index(inplace=True  )
    #data_.drop("index" , axis=1 , inplace=True)
    data_ =data_.drop(  [ "open_time"  , "close_time"   ], axis=1)


        #data_.drop("index" , axis=1 , inplace=True)
    #data_ =data_.drop(  [ "open_time"  , "close_time"  ], axis=1)
    X_train , X_test , Y_train ,   Y_test = X_y_train_test(0.8 , data_)

    oversample = SMOTE(random_state=42)
    initial_columns = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume','num_trades','taker_base_vol','taker_quote_vol', 'ignore']

    X_train, Y_train = oversample.fit_resample(X_train, Y_train)
    X_test ,Y_test = oversample.fit_resample(X_test, Y_test )
    #X_train = X_train[initial_columns]
    #X_test = X_test[initial_columns]
    # get the data from Binance  
    # klines = client.get_historical_klines(symbol, client.KLINE_INTERVAL_30MINUTE, "5 day ago")

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 

    parameters = {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 30, 'criterion': 'gini', 'bootstrap': False}

    rf = RandomForestClassifier( random_state=42 , bootstrap=parameters["bootstrap"], criterion=parameters["criterion"], 
                                        max_depth=parameters['max_depth'], max_features=parameters['max_features'], n_estimators=parameters["n_estimators"]) 

    rf.fit(X_train,Y_train)
    Y_pred = rf.predict( X_test)
        
    metrics =  { "Accuracy": accuracy_score(Y_pred, Y_test) , "Precision" : precision_score(Y_pred, Y_test , average='macro') , "Recall" : recall_score(Y_pred, Y_test , average="macro"), 
                    "F1" : f1_score(Y_pred, Y_test, average="macro")  }
    return metrics

{'Accuracy': 0.503177966101695,
 'Precision': 0.503177966101695,
 'Recall': 0.5270848954469599,
 'F1': 0.45122819999477937}

In [56]:
metrics_data = [ ]

for  percentage in [0.02 , 0.03, 0.04,0.05 ]:
    for test in [4,6,8,10,12]:

        model_params = {"periods":[ 5, 20 ,30 ],
        "time_periods" : [5,20,30],
        "periods" :[5,10,20],

        "percentage" : percentage,
        "space" : 1,
         "test" : test }

        df = get_df("ETHUSDT",model_params)

        metrics = estimate_metrics(model_params , df)
        metrics["test"]= test
        metrics["percentage"] = percentage
        meetrics_data = metrics_data + [ metrics]
pd.DataFrame(metrics_data)
    
    

KeyError: 'test'

In [98]:
validation = pd.read_csv("5last_days_klines")
predictions = rf.predict_proba(validation.drop(["open_time" , "close_time"], axis=1))

In [99]:
import numpy as np
# For ploting 
df_plot = validation
df_plot["proba_1"]=[ prediction_[0] for prediction_ in  predictions ] 
df_plot["proba_2"]=[ prediction_[1] for prediction_ in  predictions ] 
df_plot["proba_3"]=[ prediction_[2] for prediction_ in  predictions ] 


df_plot["perdictions1"]=np.where( df_plot["proba_1"] >0.80 ,1,0) 
df_plot["perdictions2"]=np.where( df_plot["proba_2"] >0.1 ,1,0) 
df_plot["perdictions3"]=np.where( df_plot["proba_3"] >0.1 ,1,0) 
 

import plotly.graph_objects as go 
from plotly.subplots import make_subplots 
 
 
df_plot["@timestamp"] = df_plot.index
 
 

fig = make_subplots(specs=[[{"secondary_y": True}]]) 
 
 

 
fig.add_trace( 
    go.Scatter(x=df_plot["@timestamp"], y=df_plot["close"], name="close_price"), 
    secondary_y=False, 
) 
 


""" fig.add_trace( 
     go.Scatter(x=df_plot["@timestamp"], y=df_plot["proba_1"], name="label"), 
     secondary_y=True, 
 ) """
fig.add_trace( 
     go.Scatter(x=df_plot["@timestamp"], y=df_plot["proba_3"], name="short"), 
     secondary_y=True, 
 ) 
fig.add_trace( 
     go.Scatter(x=df_plot["@timestamp"], y=df_plot["proba_2"], name="long"), 
     secondary_y=True, 
 ) 

fig.add_trace( 
     go.Scatter(x=df_plot["@timestamp"], y=df_plot["proba_1"] *0.3, name="hold"), 
     secondary_y=True, 
 ) 

 
# Add figure title 
fig.update_layout( 
    title_text="Predicción de tiempos elevados" 
) 

fig.update_yaxes(range=[0, 0.3], secondary_y=True)
fig.update_yaxes(range=[1150, 1300], secondary_y=False)


 
# Set x-axis title 
fig.update_xaxes(title_text="") 
 
# Set y-axes titles 
fig.update_yaxes(title_text="<b>tiempo de respuesta</b> (ms)", secondary_y=False ) 
fig.update_yaxes(title_text="<b>predicción</b> Error", secondary_y=True) 
fig.update_layout(plot_bgcolor = "white") 
 
 
fig.show()